# NanoPyx "Codeless" Jupyter Notebook

This notebook allows you to perform channel registration of 2D multi-channel image stacks

To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step.
When running this notebook on Colab, it will automatically prompt you to connect to your google Drive, from where you can select the input data.
To save the output of this notebook, make sure to select the "save output" option. Output will be saved on the same directory as the loaded image. If using an example dataset it will be saved on the path Python is currently running (if locally running the notebook) or on Colab files (if running on Colab, files button on the left bar).
If you found this work useful for your research please consider citing our [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1)  and out Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398)

**Channel Registration** Romain F Laine et al 2019 J. Phys. D: Appl. Phys. 52 163001




In [ ]:
#@title Fix numpy version in Google Colab, can be skipped if running in local environment. Session will now restart automatically. You can then proceed to the next cell.
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q numpy==1.26.4
    !pip install -q mako==1.3.0

    print("Session will now restart automatically. You can then proceed to the next cell.")
    import os
    os.kill(os.getpid(), 9)




In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q "nanopyx[colab]"
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')
else:
    !pip install -q "nanopyx[jupyter]"

import io
import os
import cv2 as cv
import skimage
import nanopyx
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

try:
    from ezinput import EZInput as EasyGui
except ImportError:
    from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]




In [ ]:
#@title Load image stack
# Create a GUI
gui_data = EasyGui("Data Loader")
global own_data
own_data = True

def on_button_select_own(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_file_upload("upload")
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data_own", description="Load data")
    gui_data["load_data_own"].on_click(on_button_load_data_clicked)
    gui_data.show()

def on_button_select_example(b):
    clear_output()
    gui_data.add_label(value="Select data to use:")
    gui_data.add_dropdown("data_source", options=image_files,
                    value="Example dataset: "+example_datasets[4], remember_value=True)
    gui_data.add_dropdown("cmaps", description="Colormap:",
                          options=sorted(list(mpl.colormaps)),
                          value="viridis", remember_value=True)
    gui_data.add_button("load_data", description="Load data")
    gui_data["load_data"].on_click(on_button_load_data_clicked_example)
    gui_data.show()

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = True
    # disable button
    gui_data["load_data_own"].disabled = True
    gui_data["load_data_own"].description = "Loading..."
    dataset_original = tiff.imread(gui_data["upload"].selected)
    gui_data["load_data_own"].disabled = False
    gui_data["load_data_own"].description = "Load data"
    gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, colormap=gui_data["cmaps"].value, continuous_update=True),)

def on_button_load_data_clicked_example(b):
    clear_output()
    gui_data.show()
    global dataset_original
    global own_data
    own_data = False
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."

    if gui_data["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)
    else:
        dataset_original = skimage.io.imread(gui_data["data_source"].value)
        gui_data._main_display.children = gui_data._main_display.children + (stackview.slice(dataset_original, continuous_update=True, colormap=gui_data["cmaps"].value),)

    # enable button
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    gui_data.save_settings()

gui_data.add_button("use_own_data", description="Use Own data")
gui_data["use_own_data"].on_click(on_button_select_own)
gui_data.add_button("use_example_data", description="Use Example data")
gui_data["use_example_data"].on_click(on_button_select_example)
gui_data.show()




# Channel Registration Parameters:  

Requires an image stack with shape: (channel, rows, columns).  

- **Reference Channel:** Which channel to be used as reference.  
- **Max Expected Shift:** Maximum amount of expected shift between channels, in pixels.  
- **Blocks per Axis:** As channel misalignmnet is not always homogeneous across the field of view, shift can be calculated for individual blocks of the field of view. This parameters sets how many blocks are created along both axis.  
- **Minimum Similarity:** Since smaller blocks may lead to shift calculation in areas of the image without any cells, minimum similarity can be used to define the minimum Pearson's Correlation Coefficient, between two blocks of different channels, required to use the calculated shifts as part of the registration. 




In [ ]:
#@title Create channel registration GUI
from nanopyx.methods import channel_registration

gui_reg = EasyGui("Channel Registration")

def on_button_register(b):
    clear_output()
    gui_reg.show()
    ref_channel = gui_reg["ref"].value
    max_shift = gui_reg["max"].value
    n_blocks = gui_reg["blocks"].value
    min_sim = gui_reg["min_sim"].value
    global dataset_registered
    gui_reg["register"].disabled = True
    gui_reg["register"].description = "Aligning..."
    if gui_reg["save"].value:
        save_translation_masks = True
        if own_data:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            save_path = path + os.sep + name + "_registered.tif"
            translation_mask_path = path + os.sep + name
        else:
            name = gui_data["data_source"].value.replace("Example dataset: ", "")
            save_path = name + "_registered.tif"
            translation_mask_path = name
    else:
        save_translation_masks = False
        translation_mask_path = ""
    dataset_registered = channel_registration.estimate_channel_registration(dataset_original,
                                                                            ref_channel,
                                                                            max_shift,
                                                                            n_blocks,
                                                                            min_sim,
                                                                            save_translation_masks=save_translation_masks,
                                                                            translation_mask_save_path=translation_mask_path,
                                                                            apply=True)
    if gui_reg["save"].value:
        tiff.imwrite(save_path, dataset_registered)
    gui_reg["register"].disabled = False
    gui_reg["register"].description = "Align"
    gui_reg._main_display.children = gui_reg._main_display.children + (stackview.slice(dataset_registered, colormap=gui_reg["cmaps"].value, continuous_update=True),)


gui_reg.add_label(value="Channel Registration parameters:")
gui_reg.add_int_slider("ref", description="Reference channel", min=0, max=dataset_original.shape[0]-1, value=0)
gui_reg.add_int_slider("max", description="Max expected drift", min=0, max=1000, value=10)
gui_reg.add_int_slider("blocks", description="Blocks per axis", min=1, max=10, value=5)
gui_reg.add_float_slider("min_sim", description="Minimum similarity", min=0, max=1, value=0.5, step=0.1)
gui_reg.add_dropdown("cmaps", description="Colormap:",
                     options=sorted(list(mpl.colormaps)),
                     value="viridis", remember_value=True)
gui_reg.add_checkbox("save", description="Save Output", value=True)
gui_reg.add_button("register", description="Register")
gui_reg["register"].on_click(on_button_register)
gui_reg.show()




## Use the following cell only if you have a previously calculated translation mask



In [ ]:
#@title Create channel registration GUI
import os
import sys

NANOPYX_INSTALLED = "nanopyx" in sys.modules
if not NANOPYX_INSTALLED:
    IN_COLAB = "google.colab" in sys.modules
    if IN_COLAB:
        !pip install -q "nanopyx[colab]"
        from google.colab import output
        output.enable_custom_widget_manager()
        from google.colab import drive
        drive.mount('/content/drive')
    else:
        !pip install -q "nanopyx[jupyter]"

import stackview
import tifffile as tiff
import matplotlib as mpl
from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.methods.channel_registration import apply_channel_registration
from IPython.display import display, clear_output

gui_reg_apply = EasyGui("Channel Registration Apply")

def on_button_apply(b):
    clear_output()
    gui_reg_apply.show()

    gui_reg_apply["Register Image"].disabled = True
    gui_reg_apply["Register Image"].description = "Aligning..."

    translation_mask_path = gui_reg_apply["upload"].selected
    img_path = gui_reg_apply["upload image"].selected

    aligned_image = apply_channel_registration(tiff.imread(img_path), tiff.imread(translation_mask_path))

    if gui_reg_apply["save"].value:
        path = gui_reg_apply["upload image"].selected_path
        name = gui_reg_apply["upload image"].selected_filename.split(".")[0]
        save_path = path + os.sep + name + "_registered.tif"
        tiff.imwrite(save_path, aligned_image)



    gui_reg_apply["Register Image"].disabled = False
    gui_reg_apply["Register Image"].description = "Align"
    gui_reg_apply._main_display.children = gui_reg_apply._main_display.children + (stackview.slice(aligned_image, colormap=gui_reg_apply["cmaps"].value, continuous_update=True),)

gui_reg_apply.add_label(value="Load translation mask:")
gui_reg_apply.add_file_upload("upload")
gui_reg_apply.add_label(value="Load image to register:")
gui_reg_apply.add_file_upload("upload image")
gui_reg_apply.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_reg_apply.add_checkbox("save", description="Save Output", value=True)
gui_reg_apply.add_button("Register Image", description="Register Image")
gui_reg_apply["Register Image"].on_click(on_button_apply)
gui_reg_apply.show()

